In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain_openai import OpenAI
import os


## Get environ variables

In [2]:
load_dotenv(find_dotenv(), override=True)

#os.environ.get('OPENAI_API_KEY')

True

## Chat Models with GPT3.5 and GPT4

In [12]:
#from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI() #this will use default gpt-3.5-turbo and temperature 0.7
#llm = ChatOpenAI(model='gpt-4-turbo-preview')

output = llm.invoke('Explain quantum computing in one sentence.')
print(output.content)

Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data at speeds unattainable by classical computers.


In [ ]:
help(ChatOpenAI)

In [14]:
from langchain.schema import(
    SystemMessage,
    AIMessage,
    HumanMessage
)

messages = [
    SystemMessage(content="You are a physicist.  Respond only in german."),
    HumanMessage(content="Explain quantum computing in one sentence.")
]

output = llm.invoke(messages)
print(output.content)

Quantum Computing nutzt die Prinzipien der Quantenmechanik, um Datenverarbeitungsaufgaben viel schneller als klassische Computer zu lösen.


## Caching LLM Responses

### In Memory-Cache

In [17]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model='gpt-3.5-turbo-instruct')

In [23]:
%%time

from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

prompt = 'Tell me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: user 9.38 ms, sys: 3.85 ms, total: 13.2 ms
Wall time: 558 ms


' \n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing!'

In [24]:
%%time
llm.invoke(prompt)

CPU times: user 521 µs, sys: 1.09 ms, total: 1.61 ms
Wall time: 1.7 ms


' \n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing!'

### SQLlite Cachain

In [29]:
%%time
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path='resources/cache.db'))

# First request (not in cache, will take longer)
llm.invoke('tell me a joke in italian')

# Second request  (cached, faster)
llm.invoke('tell me a joke in italian')

CPU times: user 6.51 ms, sys: 4.45 ms, total: 11 ms
Wall time: 8.98 ms


'\n\nPerché i pomodori non possono mai essere tristi?\n\nPerché sempre sono "pomodori" e mai "patatini"!'

## Streaming

In [32]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI()


prompt = 'write a rock song about moon and a raven'

print(llm.invoke(prompt).content)


Verse 1:
Under the pale moonlight, a raven takes flight
Across the darkened sky, soaring high
Its black wings spread wide, a messenger of the night
Calling out to the moon, a haunting cry

Chorus:
Moon and raven, dancing in the night
Their spirits intertwined, a mystical sight
Silent watchers of the world, hidden from plain sight
Moon and raven, guardians of the night

Verse 2:
The moon's silver glow, illuminates the land below
The raven's piercing eyes, see what others cannot find
Together they roam, in the shadows unknown
A bond unbreakable, of a different kind

Chorus:
Moon and raven, dancing in the night
Their spirits intertwined, a mystical sight
Silent watchers of the world, hidden from plain sight
Moon and raven, guardians of the night

Bridge:
In the darkness they find solace, in each other's presence
A cosmic connection, beyond mere mortal essence
They watch over us, in the stillness of the night
Moon and raven, forever in flight

Chorus:
Moon and raven, dancing in the night
T

In [ ]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)


# Prompt Templates

In [36]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = '''You are a experienced virologist.  Write a few sentences about the following virus {virus} in {language}.
'''

prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus='SARS-CoV-2', language='french')

prompt

'You are a experienced virologist.  Write a few sentences about the following virus SARS-CoV-2 in french.\n'

In [37]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)

print(output.content)

Le virus SARS-CoV-2, responsable de la maladie COVID-19, est un virus à ARN de la famille des coronavirus. Il a émergé en Chine en décembre 2019 et s'est rapidement propagé à travers le monde, provoquant une pandémie. Ce virus peut entraîner des symptômes légers à sévères, notamment des difficultés respiratoires et une pneumonie. Il est essentiel de suivre les mesures de prévention telles que le port du masque, la distanciation sociale et le lavage fréquent des mains pour limiter sa propagation.


## ChatPromptTemplates

In [43]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage


chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in json format'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population')
    ]
)

messages = chat_template.format_messages(n=5, area='World')

print(messages)

[SystemMessage(content='You respond only in json format'), HumanMessage(content='Top 5 countries in World by population')]


In [45]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

output = llm.invoke(messages)
print(output.content)


{
    "1": "China",
    "2": "India",
    "3": "United States",
    "4": "Indonesia",
    "5": "Pakistan"
}


# Simple Chains

In [51]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain


llm = ChatOpenAI()

template = '''You are a experienced virologist.  Write a few sentences about the following virus {virus} in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm, 
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus': 'SARS-CoV-2', 'language': 'french'})




> Entering new LLMChain chain...
Prompt after formatting:
You are a experienced virologist.  Write a few sentences about the following virus SARS-CoV-2 in french.

> Finished chain.


In [52]:
print(output)

{'virus': 'SARS-CoV-2', 'language': 'french', 'text': "Le SARS-CoV-2 est un virus de la famille des coronavirus qui a émergé en Chine en décembre 2019. Il est responsable de la maladie respiratoire COVID-19, qui peut entraîner des symptômes légers à graves, voire la mort chez les personnes vulnérables. Ce virus a provoqué une pandémie mondiale en 2020, mettant à rude épreuve les systèmes de santé et entraînant des mesures de confinement et de distanciation sociale à l'échelle internationale."}


In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain


llm = ChatOpenAI()

template = 'What is the capital of {country}? List the top 3 places to visit in that city. Use bullet points.'
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm, 
    prompt=prompt_template,
    verbose=True
)

country = input('Enter a country: ')
output = chain.invoke({'country': country})
print(output['text'])


## Sequential Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5)
prompt_template1 = PromptTemplate.from_template(
    template = 'You are an experienced scientist and Python programmer.  Write a function that implements the concept of {concept}.'
)

chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

llm2 = ChatOpenAI(model='gpt-4-turbo-preview', temperature=1.2)

prompt_template2 = PromptTemplate.from_template(
    template = 'Given the the Python function {function}, describe it as detailed as possible'
)

chain2 = LLMChain(llm=llm2, prompt=prompt_template2)    


overall_chain = SimpleSequentialChain(chains = [chain1, chain2], verbose=True)

output = overall_chain.invoke('linear regression')

In [ ]:
print(output['output'])

## Langchain Agents in Action: Python REPL
### Read-Eval-Print Loop

In [7]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()

python_repl.run('print([n for n in range (1,100) if n % 13 == 0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [11]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

agent_executor = create_python_agent(
    llm=llm, 
    tool=PythonREPLTool(), 
    verbose=True
)

agent_executor.invoke('Calculate the square root of the factorial 12 and display it with 4 decimal points')




> Entering new AgentExecutor chain...
To solve this, I will first calculate the factorial of 12 using the `math.factorial` function. Then, I will calculate the square root of that result using `math.sqrt`. Finally, I will format the result to display it with 4 decimal points using the `format` function.
Action: Python_REPL
Action Input: import math
Observation: 
Thought:Now that the math module is imported, I can calculate the factorial of 12.
Action: Python_REPL
Action Input: factorial_12 = math.factorial(12)
Observation: 
Thought:Now that I have calculated the factorial of 12, I will calculate its square root.
Action: Python_REPL
Action Input: sqrt_factorial_12 = math.sqrt(factorial_12)
Observation: 
Thought:Now that I have the square root of the factorial of 12, I will format it to display with 4 decimal points.
Action: Python_REPL
Action Input: print(f"{sqrt_factorial_12:.4f}")
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

> Finished chain

{'input': 'Calculate the square root of the factorial 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [14]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [15]:
response

{'input': 'What is the answer to 5.1 ** 7.3', 'output': '146306.05007233328'}

In [16]:
# this the value of response:
# {'input': 'What is the answer to 5.1 ** 7.3', 'output': '146306.05007233328'}

print(response['input'])

print(response['output'])

What is the answer to 5.1 ** 7.3
146306.05007233328


## Langchain Tool: DuckDuckGo and Wikipedia

In [19]:
from langchain.tools import DuckDuckGoSearchRun 

search = DuckDuckGoSearchRun()

output = search.invoke('Where was Freddie Mercury born?')

print(output)


Freddie Mercury was born Farrokh Bulsara in Stone Town, in the British protectorate of Zanzibar (now part of Tanzania), on September 5, 1946. His first big challenge was to come to terms with... But there are many lesser-known Freddie Mercury facts that reveal hidden depths to the life and work of the man born Farrokh Bulsara in Zanzibar, on September 5, 1946. Here are 15 surprising... Although he was born on the East African island of Zanzibar on September 5, 1946, his parents were both Indian; they were Parsees from Gujarat. Mercury would often make note of his Persian roots, but the Parsees had been driven from Persia a thousand years before. They landed on the Gujarat coast and populated its greatest city, Bombay. Table of Contents Freddie Mercury was born on September 5, 1946. Freddie Mercury, whose real name was Farrokh Bulsara, was born in Zanzibar, Tanzania. He later became the lead vocalist of the legendary rock band Queen. Freddie Mercury had a four-octave vocal range. Freddi

In [20]:
search.name

'duckduckgo_search'

In [21]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

In [22]:
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

output = search.run('Freddie Mercury and Queen')

print(output)

[snippet: From his earliest songs with Queen, Mercury built a reputation as one of the most popular songwriters of his generation, and the best Freddie Mercury songs reveal an artist committed to..., title: Best Freddie Mercury Songs: 20 Essential Solo And Queen Tracks, link: https://www.udiscovermusic.com/stories/best-freddie-mercury-songs/], [snippet: Everybody knows that Freddie Mercury was a gifted songwriter, a breathtakingly original performer, and frontman of one of the greatest bands of all time, Queen. But there are many..., title: Freddie Mercury Facts: Things You Never Knew About Queen's Frontman, link: https://www.udiscovermusic.com/stories/freddie-mercury-facts-you-need-to-know/], [snippet: Ex-fiancée explains decision Who is Mary Austin? Meet the woman Freddie Mercury asked to marry Freddie Mercury facts: Queen singer's age, teeth, real name, relationships and more explained Freddie Mercury in private: 20 rarely seen photos of the star behind closed doors, title: Freddie 

In [24]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region = 'de-de', max_results = 3, safesearch = 'moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source = 'news')

output=search.run('Berlin')

print(output)
